In [12]:
import pandas as pd
import numpy as np


users=pd.read_csv('data/users_f_location_1.1.csv')
books=pd.read_csv('data/books_1.0.csv')
ratings=pd.read_csv('data/train_ratings.csv')
submission_test=pd.read_csv('data/sample_submission.csv')
datasets=ratings.merge(users, on='user_id', how='left').merge(books, on='isbn', how='left')

dictionary=pd.concat([ratings,submission_test])
user2idx = {v:k for k,v in enumerate(dictionary['user_id'].unique())}
book2idx = {v:k for k,v in enumerate(dictionary['isbn'].unique())}

In [13]:
users.head()

,user_id,age,location_city,location_state,location_country
0,8,NaN,timmins,ontario,canada
1,11400,49.0,ottawa,ontario,canada
2,67544,30.0,toronto,ontario,canada
3,85526,36.0,victoria,britishcolumbia,canada
4,96054,29.0,ottawa,ontario,canada


- age: 결측치인 유저가 읽은 책들을 보고 그 책들을 읽은 유저의 평균으로 결측치 채움
- location_country: 8개의 카테고리로 변환
    - 1: 남아메리카
    - 2: 북아메리카
    - 3: 유럽
    - 4: 오세아니아
    - 5: 아시아(서북)
    - 6: 아시아(동남)
    - 7: 중동
    - 8: 아프리카

In [14]:
#책의 isbn을 키로가지고 그에따른 평균유저나이를 나타내는 딕셔너리
isbn_age=datasets.groupby('isbn').mean()['age'].dropna().to_dict()

In [15]:
#전체 데이터셋에서 나이가 결측치인 부분만 채우기
datasets['newage']=np.where(datasets['age'].isna(),datasets['isbn'].map(isbn_age),datasets['age'])
#datasets이용해 유저아이디별로 나이
userid_age=datasets.groupby('user_id').mean()['newage'].to_dict()

In [16]:
users['age']=users['user_id'].map(userid_age)

In [17]:
#나머지 결측치 location평균으로 채우면 성능더떨어짐
#loca_age=users.groupby('location_state').mean()['age'].to_dict() 

#users['age']=np.where(users['age'].isna(),users['location_country'].map(loca_age),users['age'])


users.drop(['location_state','location_city'],axis=1,inplace=True)

In [18]:
cntr_cate={
    'usa': 2, 'canada': 2, 'unitedkingdom': 3, 'germany': 3, 'spain': 3, 'australia':4, 'france': 3, 'portugal': 3, 'malaysia': 5, 
    'netherlands': 3, 'switzerland': 3, 'newzealand': 4, 'italy': 3, 'austria': 3, 'singapore' :6, 'philippines': 6, 'romania': 3, 
    'iran':7 , 'brazil':1, 'belgium': 3, 'dominicanrepublic':1 , 'sweden': 3, 'finland': 3, 'japan':6, 'china':6, 'england': 3, 
    'norway': 3, 'argentina':1, 'mexico':1 , 'poland': 3, 'denmark': 3, 'ireland': 3, 'newyork':2, 'israel':7, 'caymanislands': 3, 
    'southafrica':8, 'hongkong':6, 'us':2 , 'india':6, 'taiwan':6, 'aruba':3, 'kuwait':7,'phillipines':6, 'southkorea':6, 'okinawa':6, 
    'bulgaria':3
}

users['location_country']=users['location_country'].map(cntr_cate)

In [19]:
books.head()

,isbn,book_title,book_author,year_of_publication,publisher,img_url,language,category,summary,img_path
0,0002005018,Clara Callan,Richard Bruce Wright,2001.0,0,http://images.amazon.com/images/P/0002005018.0...,0,['Actresses'],"In a small town in Canada, Clara Callan reluct...",images/0002005018.01.THUMBZZZ.jpg
1,0060973129,Decision in Normandy,Carlo D'Este,1991.0,6,http://images.amazon.com/images/P/0060973129.0...,0,['1940-1949'],"Here, for the first time in paperback, is an o...",images/0060973129.01.THUMBZZZ.jpg
2,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999.0,374,http://images.amazon.com/images/P/0374157065.0...,0,['Medical'],"Describes the great flu epidemic of 1918, an o...",images/0374157065.01.THUMBZZZ.jpg
3,0399135782,The Kitchen God's Wife,Amy Tan,1991.0,399,http://images.amazon.com/images/P/0399135782.0...,0,['Fiction'],A Chinese immigrant who is convinced she is dy...,images/0399135782.01.THUMBZZZ.jpg
4,0425176428,What If?: The World's Foremost Military Histor...,Robert Cowley,2000.0,425,http://images.amazon.com/images/P/0425176428.0...,0,['History'],"Essays by respected military historians, inclu...",images/0425176428.01.THUMBZZZ.jpg


카테고리 숫자 10개로 나누기

In [20]:
category_list=books['category'].unique().tolist()
cate_dict={}
cnt=0
for i in range(len(category_list)):
    if category_list[i]==-1:
        cate_dict[category_list[i]]=-1

    elif ('fiction' in str(category_list[i]).lower()) or ('fictitious' in str(category_list[i]).lower()) or ('act' in str(category_list[i]).lower()) \
    or ('adolescence' in str(category_list[i]).lower()) or ('humor' in str(category_list[i]).lower()) or ('travel' in str(category_list[i]).lower()) \
    or ('poetry' in str(category_list[i]).lower()) or ('literary criticism' in str(category_list[i]).lower()) or ('drama' in str(category_list[i]).lower()) \
    or ('comic' in str(category_list[i]).lower()) or ('stories' in str(category_list[i]).lower()) or ('story' in str(category_list[i]).lower()) \
    or ('literary' in str(category_list[i]).lower()) or ('brother' in str(category_list[i]).lower()) or ('cats' in str(category_list[i]).lower())\
    or ('england' in str(category_list[i]).lower()) or ('australia' in str(category_list[i]).lower()) or ('britain' in str(category_list[i]).lower())\
    or ('america' in str(category_list[i]).lower()) or ('adventure' in str(category_list[i]).lower()) or ('children' in str(category_list[i]).lower())\
    or ('fantasy' in str(category_list[i]).lower()) or ('horror' in str(category_list[i]).lower()) or ('dog' in str(category_list[i]).lower())\
    or ('france' in str(category_list[i]).lower()) or ('tale' in str(category_list[i]).lower()) or ('united states' in str(category_list[i]).lower()) \
    or ('canada' in str(category_list[i]).lower()) or ('authors' in str(category_list[i]).lower()):
        cate_dict[category_list[i]]=8

    elif ('psychology' in str(category_list[i]).lower()) or ('philosophy' in str(category_list[i]).lower()):
        cate_dict[category_list[i]]=1

    elif ('religion' in str(category_list[i]).lower()) or ('jews' in str(category_list[i]).lower()) or ('bible' in str(category_list[i]).lower())\
        or ('christian' in str(category_list[i]).lower()):
        cate_dict[category_list[i]]=2

    elif ('social' in str(category_list[i]).lower()) or ('econom' in str(category_list[i]).lower()) or ('mind' in str(category_list[i]).lower()) \
        or ('famil' in str(category_list[i]).lower()) or ('self-help' in str(category_list[i]).lower()) or ('law' in str(category_list[i]).lower()) \
        or ('crime' in str(category_list[i]).lower()) or ('education' in str(category_list[i]).lower()) or ('friendship' in str(category_list[i]).lower()) \
        or ('life' in str(category_list[i]).lower()) or ('murder' in str(category_list[i]).lower()):
        cate_dict[category_list[i]]=3

    elif ('computer' in str(category_list[i]).lower()) or ('cooking' in str(category_list[i]).lower()) or ('reference' in str(category_list[i]).lower()) \
        or ('gardening' in str(category_list[i]).lower()) or ('medical' in str(category_list[i]).lower()) or ('technology' in str(category_list[i]).lower()) \
        or ('house' in str(category_list[i]).lower()) or ('transportation' in str(category_list[i]).lower()):
        cate_dict[category_list[i]]=5

    elif ('fitness' in str(category_list[i]).lower()) or ('art' in str(category_list[i]).lower()) or ('music' in str(category_list[i]).lower()) \
        or ('sports' in str(category_list[i]).lower()) or ('hobb' in str(category_list[i]).lower()) or ('photo' in str(category_list[i]).lower()) \
        or ('architecture' in str(category_list[i]).lower()) or ('game' in str(category_list[i]).lower()) or ('antiques' in str(category_list[i]).lower())\
        or ('design' in str(category_list[i]).lower()):
        cate_dict[category_list[i]]=6

    elif ('language' in str(category_list[i]).lower()):
        cate_dict[category_list[i]]=7

    elif ('nature' in str(category_list[i]).lower()) or ('science' in str(category_list[i]).lower()) or ('pet' in str(category_list[i]).lower()) \
        or ('animal' in str(category_list[i]).lower()) or ('planet' in str(category_list[i]).lower()) or ('math' in str(category_list[i]).lower()) \
        or ('dinosaur' in str(category_list[i]).lower()):
        cate_dict[category_list[i]]=4
        
    elif ('biography' in str(category_list[i]).lower()) or ('histor' in str(category_list[i]).lower()) or ('1940-' in str(category_list[i]).lower()):
        cate_dict[category_list[i]]=9

    elif ('biography' in str(category_list[i]).lower()):
        cate_dict[category_list[i]]=9

In [21]:
books['category']=books['category'].map(cate_dict)

In [22]:
books.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 149570 entries, 0 to 149569
Data columns (total 10 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   isbn                 149570 non-null  object 
 1   book_title           149570 non-null  object 
 2   book_author          149570 non-null  object 
 3   year_of_publication  149570 non-null  float64
 4   publisher            149570 non-null  int64  
 5   img_url              149570 non-null  object 
 6   language             149570 non-null  int64  
 7   category             74376 non-null   float64
 8   summary              82343 non-null   object 
 9   img_path             149570 non-null  object 
dtypes: float64(2), int64(2), object(6)
memory usage: 11.4+ MB


남은 카테고리는 유저들이 읽은 다른책이용해서 카테고리 채우기

In [23]:
datasets2=datasets.copy()
datasets2['category']=datasets['category'].map(cate_dict)
user_cate=datasets2.groupby('user_id').mean()['category'].dropna().apply(lambda x:int(x)).to_dict()
datasets2['category']=np.where(datasets2['category'].isna(),datasets2['user_id'].map(user_cate),datasets2['category'])
isbn_cate=datasets2.groupby('isbn').mean()['category'].dropna().to_dict()
books['category']=np.where(books['category'].isna(),books['isbn'].map(isbn_cate),books['category'])
books.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 149570 entries, 0 to 149569
Data columns (total 10 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   isbn                 149570 non-null  object 
 1   book_title           149570 non-null  object 
 2   book_author          149570 non-null  object 
 3   year_of_publication  149570 non-null  float64
 4   publisher            149570 non-null  int64  
 5   img_url              149570 non-null  object 
 6   language             149570 non-null  int64  
 7   category             130751 non-null  float64
 8   summary              82343 non-null   object 
 9   img_path             149570 non-null  object 
dtypes: float64(2), int64(2), object(6)
memory usage: 11.4+ MB


In [24]:
books.to_csv('books_2.0.csv',index=False)
users.to_csv('users_2.0.csv',index=False)